In [1]:
### Importing necessary libraries
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import xlwt 
from xlwt import Workbook 

In [2]:
### Data preparation
class Concrete:
    
    def __init__(self, IMG_HEIGHT, IMG_WIDTH):
        self.IMG_HEIGHT = IMG_HEIGHT
        self.IMG_WIDTH  = IMG_WIDTH
        self.POS = "data_sets/concrete/Positive"
        self.NEG = "data_sets/concrete/Negative"
        self.TESTING = "PetImages/Testing"
        self.LABELS = {self.POS: 0, self.NEG: 1}
        self.all_available_data = []
        self.poscount = 0
        self.negcount = 0
   

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_HEIGHT, self.IMG_WIDTH))
                        self.all_available_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])  # do something like print(np.eye(2)[1]), just makes one_hot 
                        #print(np.eye(2)[self.LABELS[label]])

                        if label == self.POS:
                            self.poscount += 1
                        elif label == self.NEG:
                            self.negcount += 1

                    except Exception as e:
                        pass
                        #print(label, f, str(e))

        np.random.shuffle(self.all_available_data)
        np.save("all_available_data_concrete.npy", self.all_available_data)
        print('Positive:',self.poscount)
        print('Negative:',self.negcount)



In [3]:
### Set the Neural Net
class Net(nn.Module):
    
    def __init__(self, IMG_HEIGHT, IMG_WIDTH):
        '''
        First inherit the nn.module class to use pytorch.
        Then add the convulutional layers.
        Do the flattening.
        Then liner/dense layerse can be added accordingly.
        '''
        super().__init__() # just run the init of parent class (nn.Module)
        
        self.IMG_HEIGHT = IMG_HEIGHT
        self.IMG_WIDTH  = IMG_WIDTH
        
        self.conv1 = nn.Conv2d(1, 32, 5) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.conv2 = nn.Conv2d(32, 64, 5) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 kernel / window
        self.conv3 = nn.Conv2d(64, 128, 5)
   
        x = torch.randn(self.IMG_HEIGHT,self.IMG_WIDTH).view(-1,1,self.IMG_HEIGHT,self.IMG_WIDTH) # A random tensor is passed through the conv. layers once so that
        self._to_linear = None                  # the size of the output of the last conv. layer can be found.
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512) #flattening.
        self.fc2 = nn.Linear(512, 2) # 512 in, 2 out bc we're doing 2 classes (dog vs cat).
        

    def convs(self, x):
        '''
        Applying max_pooling and using appropiate activation function for conv. layers
        '''
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    

    def forward(self, x):
        '''
        Reshaping after conv. layers accordingly 
        Then applying activation functions to the linear layers and the output layers.
        '''
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        
        return F.softmax(x, dim=1)

In [4]:
class Model:
    
    def __init__(self, MODEL_NAME,  BATCH_SIZE, EPOCHS, test_X, test_y, testing_size, IMG_HEIGHT, IMG_WIDTH):
        self.BATCH_SIZE = BATCH_SIZE
        self.EPOCHS = EPOCHS
        self.test_X = test_X
        self.test_y = test_y
        self.IMG_HEIGHT = IMG_HEIGHT
        self.IMG_WIDTH  = IMG_WIDTH
        self.size = testing_size        
        self.MODEL_NAME = MODEL_NAME
    
    def fwd_pass(self, X, y, train=False):
        '''
        Passing the data for both training and testing
        '''
        if train:
            net.zero_grad()
        outputs = net(X)
        matches  = [torch.argmax(i)==torch.argmax(j) for i, j in zip(outputs, y)]
        acc = matches.count(True)/len(matches)
        loss = loss_function(outputs, y)

        if train:
            loss.backward()
            optimizer.step()

        return acc, loss
    
    
    def fit(self, net, train_X, train_y):
        '''
        Train the data, set HYPER_PARAMETERS(Epocsh, batch_size).
        Set the optimizer    
        '''
        wb = Workbook()
        sheet_name = self.MODEL_NAME 
        s1 = wb.add_sheet(sheet_name)
        s1.write(0,0,'Training Accuracy')
        s1.write(0,1,'Test Accuracy')
        s1.write(0,2,'Training Loss')
        s1.write(0,3,'Test Loss')
        for epoch in range(self.EPOCHS):
            for i in tqdm(range(0, len(train_X), self.BATCH_SIZE)):
                batch_X = train_X[i:i+self.BATCH_SIZE].view(-1, 1, IMG_HEIGHT, IMG_WIDTH)
                batch_y = train_y[i:i+self.BATCH_SIZE]

                batch_X, batch_y = batch_X.to(device), batch_y.to(device)

                acc, loss = self.fwd_pass(batch_X, batch_y, train=True)
            
            random_start = np.random.randint(len(self.test_X)-self.size)
            X, y = self.test_X[random_start:random_start+self.size], self.test_y[random_start:random_start+self.size]
            with torch.no_grad():
                val_acc, val_loss = self.fwd_pass(X.view(-1, 1, IMG_HEIGHT, IMG_WIDTH).to(device), y.to(device))
            print(f"Epoch  {epoch+1} :\n")
            print(' Training Accuracy :', acc, '\n', 'Test Accuracy :', val_acc, '\n', 'Training Loss :', loss.item(), '\n', 'Test Loss :', val_loss.item())   
            s1.write(epoch+1,0, round(float(acc),2))
            s1.write(epoch+1,1, round(float(val_acc),2))
            s1.write(epoch+1,2, round(float(loss), 4))
            s1.write(epoch+1,3, round(float(val_loss), 4))
            filename = self.MODEL_NAME +'.xls'
            wb.save(filename) 
            
            
    def predict(self, net, X_pred):
        X_pred = X_pred.view(-1, 1, IMG_HEIGHT, IMG_WIDTH)
        X_pred = X_pred.to(device)
        outputs = net(X_pred)
        prediction = torch.argmax(outputs)
        if prediction == 0:
            print('There is a crack')
        else:
            print('There is no crack')        

In [ ]:
##### Main Code Here #####
    
# Loading the data, if data is already prepared once, just load it. Once the data is prepared, save it and turn the flag to False
REBUILD_DATA = True # set to true to one once, then back to false unless you want to change something in your training data.

IMG_HEIGHT = 100 # Set the image size IMG_HEIGHT x IMG_WIDTH pixel
IMG_WIDTH  = 100
if REBUILD_DATA:
    concrete = Concrete(IMG_HEIGHT, IMG_WIDTH)
    concrete.make_training_data()
    
all_available_data = np.load("all_available_data_concrete.npy", allow_pickle=True)

### Define a device to start GPU
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

### Connecting the Neural Net to the GPU. Choosing the loss function, and optimizer parameters
LEARNING_RATE = 0.001
net = Net(IMG_HEIGHT, IMG_WIDTH).to(device)
loss_function = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
print('Neural Net Features : ',net)
print('Optimizer Features : ',optimizer)
print('Loss Function : ', loss_function)
    
### Seperating features and labels, normalizing the data
X = torch.Tensor([i[0] for i in all_available_data]).view(-1,IMG_HEIGHT,IMG_WIDTH)
X = X/255.0
y = torch.Tensor([i[1] for i in all_available_data])

### Splitting the data as training and testing data
VAL_PCT = 0.1  # Test set ratio
val_size = int(len(X)*VAL_PCT)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print('Size of the Data Set :', len(all_available_data))
print('Training Set Ratio [%] :', 100 * VAL_PCT)
print('Size of the Train Set :', len(train_X), ' <---> Size of the Test Set :', len(test_X))

### Training HYPERPARAMETERS
BATCH_SIZE = 32
EPOCHS = 10
testing_size =  100 # len(test_X) -1
#MODEL_NAME = f"model-{int(time.time())}"

#for batch in BATCH_SIZE:
batch = BATCH_SIZE
MODEL_NAME = f"model-Batch{int(batch)}-{int(time.time())}"  # gives a dynamic model name, to just help with things getting messy over time. 

print('\n HYPERPARAMETERS : ')
print('Batch Size : ', batch)
print('# of Epochs : ', EPOCHS)

model = Model(MODEL_NAME, batch, EPOCHS, test_X, test_y, testing_size, IMG_HEIGHT, IMG_WIDTH)
model.fit(net, train_X, train_y)

Running on the GPU
Neural Net Features :  Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=10368, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)
Optimizer Features :  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
Loss Function :  MSELoss()


  0%|                                                                                         | 0/1125 [00:00<?, ?it/s]

Size of the Data Set : 40000
Training Set Ratio [%] : 10.0
Size of the Train Set : 36000  <---> Size of the Test Set : 4000

 HYPERPARAMETERS : 
Batch Size :  32
# of Epochs :  10


  0%|▏                                                                                | 3/1125 [00:00<00:37, 29.57it/s]

Epoch  1 :

 Training Accuracy : 0.96875 
 Test Accuracy : 0.96 
 Training Loss : 0.03585147112607956 
 Test Loss : 0.033420778810977936


  0%|▏                                                                                | 3/1125 [00:00<00:46, 24.00it/s]

Epoch  2 :

 Training Accuracy : 0.9375 
 Test Accuracy : 0.99 
 Training Loss : 0.025847282260656357 
 Test Loss : 0.008279291912913322


 56%|████████████████████████████████████████████▎                                  | 631/1125 [00:29<00:22, 22.25it/s]

In [ ]:
##### Data Visualization #####
import pandas as pd

columns = ['Training Accuracy', 'Test Accuracy', 'Training Loss', 'Test Loss' ]
results_df = pd.read_excel(f"model-Batch{int(batch)}-{int(time.time())}.xls")
results_df.columns = columns
                             

In [ ]:
results_df[['Training Accuracy','Test Accuracy']].plot(grid=True)
results_df[['Training Loss','Test Loss']].plot(grid=True)

In [ ]:
"data_sets/concrete/Predict/0.jpg"

In [ ]:
path = "data_sets/concrete/Predict/0.jpg"
img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
X_pred = torch.Tensor(img).view(-1,IMG_HEIGHT,IMG_WIDTH)

In [ ]:
X_pred = X_pred/255

In [ ]:
X_pred = X_pred

In [ ]:
model = Model(MODEL_NAME, batch, EPOCHS, test_X, test_y, testing_size, IMG_HEIGHT, IMG_WIDTH)
model.predict(net, X_pred)

In [ ]:
path = "data_sets/concrete/Predict/1.jpg"
img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
X_pred = torch.Tensor(img).view(-1,IMG_HEIGHT,IMG_WIDTH)